# Covid19 Vaccionation Statistics in Maryland

In [1]:
# importing dependencies and resources
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json, requests
import gmaps
from scipy import spatial
# Import API key
from config import g_key

In [2]:
# loading csv files
vaccination_locations_path = "../Datasets/MD_Vaccination_Locations.csv"


<center><i><h1 style="background-color:black;color:orange">Total covid-19 vaccine doses administered In The US</h1></i></center>
<div style="background-color:LightSlateGray">
<ul>
    <li>
        <p style="color:white;font-size:150%;background-color:LightSlateGray"> Analysis of the total number of vaccine doses administrated.</p>
    </li>
    <li>
        <p style="color:white;font-size:150%;background-color:LightSlateGray"> We will compare Maryland against other states in the United States.</p>
    </li>
    </ul>
</div>

<center><i><h1 style="background-color:black;color:orange">All Maryland Vaccination Sites</h1></i></center>
<div style="background-color:LightSlateGray">
<ul>
    <li>
        <p style="color:white;font-size:150%;background-color:LightSlateGray"> This Section shows a map with all the vaccination sites in the State of Maryland.</p>
    </li>
    <li>
        <p style="color:white;font-size:150%;background-color:LightSlateGray"> You can Find the nearest vaccionation place to a specific address, providing it in the inputbox.</p>
    </li>
    </ul>
</div>

In [3]:
# Creating the initial DataFrame
locations_df = pd.read_csv(vaccination_locations_path)

# enable gmaps features on jupyternotebooks
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Configure gmaps key

In [4]:
# Google developer API key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
locations_df.head()

,X,Y,OBJECTID,facilityid,name,fulladdr,municipality,CreationDate,Creator,EditDate,...,X_coord,Y_coord,test_pediatric,multi_language,test_pediatric_notes,created_user,created_date,last_edited_user,last_edited_date,County
0,-76.534166,38.990512,84,Anna_Lumi_83,Luminis Health Anne Arundel Medical Center,"2001 Medical Parkway, Annapolis, MD 21401",Annapolis,1970/01/01 00:00:00+00,NaN,1970/01/01 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,mdimapdatacatalog,2021/03/02 22:05:19.604+00,mdimapdatacatalog,2021/03/02 22:05:19.604+00,Anne Arundel
1,-76.865391,38.982597,87,Lanh_Lumi_86,Luminis Health Doctors Community Medical Center,"8118 Good Luck Road, Lanham, MD 20706",Lanham,1970/01/01 00:00:00+00,NaN,1970/01/01 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,mdimapdatacatalog,2021/03/02 22:05:19.604+00,mdimapdatacatalog,2021/03/02 22:05:19.604+00,Prince George's
2,-76.648885,39.288231,92,Balt_Grac_91,Grace Medical Center (formerly Bon Secours Hos...,"2000 West Baltimore Street, Baltimore, MD 21223",Baltimore,1970/01/01 00:00:00+00,NaN,1970/01/01 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,mdimapdatacatalog,2021/03/02 22:05:19.604+00,mdimapdatacatalog,2021/03/02 22:05:19.604+00,Baltimore City
3,-76.990634,39.557869,93,West_Carr_92,Carroll Hospital Center,"200 Memorial Avenue, Westminster, MD 21157",Westminster,1970/01/01 00:00:00+00,NaN,1970/01/01 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,mdimapdatacatalog,2021/03/02 22:05:19.604+00,mdimapdatacatalog,2021/03/02 22:05:19.604+00,Carroll
4,-76.885867,39.214293,95,Colu_Howa_94,Howard County General Hospital,"5755 Cedar Lane, Columbia, MD 21044",Columbia,1970/01/01 00:00:00+00,NaN,1970/01/01 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,mdimapdatacatalog,2021/03/02 22:05:19.604+00,mdimapdatacatalog,2021/03/02 22:05:19.604+00,Howard


In [14]:
# setting up map

#store lat and lng
locations = locations_df[["Y","X"]]

#map attributes
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': 'auto'
}
vaccination_map = gmaps.figure(center = [0,0] ,zoom_level =6, layout=figure_layout)

In [7]:
# Using the template add the Vaccionation sites marks to the map
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{fulladdr}</dd>
<dt>municipality</dt><dd>{municipality}</dd>
</dl>
"""
# Store the DataFrame Row
places_info = [info_box_template.format(**row) for index, row in locations_df.iterrows()]
locations = locations_df[["Y", "X"]]

In [8]:
# Add marker layer ontop of heat map

# creating the markers
markers = gmaps.marker_layer(locations, info_box_content= places_info)

#adding the markers
vaccination_map.add_layer(markers)

# Display figure
vaccination_map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='auto', padding='1px', width='1000…

In [9]:
users_address = input("Enter your Address >> ")

Enter your Address >> Rockville MD


In [10]:
# Target city
target_address = users_address

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_address, g_key)
# sending request through Google API
geo_data = requests.get(target_url).json()

# Extract latitude and longitude
lat = geo_data["results"][0]["geometry"]["location"]["lat"]
lng = geo_data["results"][0]["geometry"]["location"]["lng"]

In [11]:
# Obtaining a list with tuples inside containing the coordinates of all the vaccination sites in Maryland
vaccination_points = [(x,y) for y,x in zip(locations['Y'], locations['X'])]
tree = spatial.KDTree(vaccination_points)
result = tree.query([(lng,lat)])
# obtaining the index that containts the nearest place using the coordinates to compare the distance
index = result[1][0] 

In [12]:

facility = locations_df.iloc[index]["name"]
address = locations_df.iloc[index]["fulladdr"]

In [13]:
print(f"The nearest vaccination center to you is: {facility} ")
print(f"located at: {address}")


The nearest vaccination center to you is: Walgreens Rockville 
located at: 430 Hungerford Dr Rockville MD 20850
